# hurricane-satellites
This notebook is where the main processing occurs

## Setup
The following cells setup and run test to make sure things will run smoothly.

In [ ]:
%cd ~/
!git clone https://github.com/hammad93/hurricane-satellites.git

In [ ]:
!pip install -r hurricane-satellites/requirements.txt

In [1]:
import os
# https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html#environment-variables
vars = ["EUMETSAT_PASS", "EUMETSAT_SECRET", "BASE64_SSH_GEOSERVER", "HOST_SSH", "USER_SSH"]
if None in [os.getenv(env) for env in vars] :
  from google.colab import userdata
  for var in vars :
    os.environ[var] = userdata.get(var)

## Create Satellite Classes
Utilizing software design patterns, we will create a satellite object based on a predefined satellite class for each satellite we are processing.

In [2]:
%cd ~/hurricane-satellites
import satellite
import goes_east
import goes_west
import h8
import msg_0
import msg_io

/root/hurricane-satellites


In [ ]:
import time
from concurrent.futures import ThreadPoolExecutor

# Your satellite sources list
satellites = [
    goes_east.GOESEastDataSource(),
    goes_west.GOESWestDataSource(),
    msg_0.MSG0DegreeDataSource(),
    msg_io.MSGIndianOceanDataSource(),
    h8.Himawari8DataSource()
]

def fetch_data_for_satellite(satellite, file_prefix):
    """
    Wrapper function to call getRecentData on a satellite object.
    """
    satellite.getRecentData(file_prefix=file_prefix)

def main():
    prefix = f"[{int(time.time())}]"
    # Use ThreadPoolExecutor to run getRecentData concurrently for each satellite
    with ThreadPoolExecutor(max_workers=len(satellites)) as executor:
        # Schedule the executions
        futures = [executor.submit(fetch_data_for_satellite, satellite, prefix) for satellite in satellites]

        # Optionally, wait for all futures to complete and handle results/errors
        for future in futures:
            try:
                # Result handling here if needed
                future.result()  # This will raise exceptions if any occurred
            except Exception as e:
                print(f"An error occurred: {e}")
main()

## Extract and Download

In [4]:
satellites

In [ ]:
for satellite in satellites :
    satellite.toNetCDF()

## Upload

In [ ]:
import base64
with open('key.pem', 'wb') as f:
  f.write(base64.b64decode(userdata.get('BASE64_SSH_GEOSERVER')))

In [19]:
import os
upload_queue = []
for f in os.listdir():
    if f[-2:] == "nc": # NetCDF file extension
        upload_queue.append(f)
print(upload_queue)

['[1713331543][H8]_2_combined_image_greyscale.nc', '[1713331543][H8]_3_combined_image_greyscale.nc', '[1713331543][GOES-18]_GOES18-ABI-FD-GEOCOLOR-10848x10848.nc', '[1713331543][H8]_1_combined_image_greyscale.nc', '[1713331543][GOES-16]_GOES16-ABI-FD-GEOCOLOR-10848x10848.nc']


In [ ]:
import pysftp
cnopts = pysftp.CnOpts()
cnopts.hostkeys = None

with pysftp.Connection(host=os.getenv('HOST_SSH'), username=os.getenv('USER_SSH'), private_key="key.pem", cnopts=cnopts) as sftp:
    with sftp.cd('/home/bitnami/hurricane-server/wms_data/'):
      for upload in upload_queue:
          print(f"Uploading {upload} . . .")
          sftp.put(upload)


## Appendix
This code is here for reference and can be safely removed.

In [23]:
!git diff

diff --git a/msg_0.py b/msg_0.py
index d1168d4..d7550da 100644
--- a/msg_0.py
+++ b/msg_0.py
@@ -16,7 +16,7 @@ class MSG0DegreeDataSource(satellite.DataSource):
         """
         self.recent_file_prefix = file_prefix
         credentials = (self.consumer_key, self.consumer_secret)
-        token = eumdac.AccessToken(credentials)
+        token = eumdac.AccessToken(credentials, cache=False)
 
         print(f"This token '{token}' expires {token.expiration}")
 
@@ -63,7 +63,7 @@ class MSG0DegreeDataSource(satellite.DataSource):
         # output to NetCDF
         output = scn.load(scn.available_dataset_names(), upper_right_corner='NE')
         scn.save_datasets(
-            filename=f"{self.recent_file_prefix}[{self.id}]",
+            filename=f"{self.recent_file_prefix}[{self.id}].nc",
             writer="cf",
             groups={
                 'default': filter(lambda x: x!='HRV', scn.available_dataset_names()),
diff --git a/msg_io.py b/msg_io.py
index 652fc92..53c5317 100